In [47]:
import cartopy.crs as ccrs
import gpxpy
import iris
import iris.plot as iplt
import iris.quickplot as qplt
import matplotlib.pyplot as plt
import os
import pandas as pd

# Define a Cartopy 'ordinary' lat-lon coordinate reference system.
crs_latlon = ccrs.PlateCarree()

In [2]:
gpx_file = '/home/jseddon/python/elinca/yblog-1589101179046.gpx'

In [7]:
with open(gpx_file) as fh:
    gpx = gpxpy.parse(fh)
data = gpx.tracks[0].segments[0].points

In [12]:
df = pd.DataFrame(columns=['lon', 'lat', 'time'])
for point in data:
    df = df.append({'lon': point.longitude, 'lat' : point.latitude, 'time' : point.time}, ignore_index=True)

In [15]:
daily = df[df.time.dt.strftime('%H:%M').between('04:00', '12:00')] 
print(daily[:10])

        lon       lat                       time
4  -4.84571  53.64186  2013-09-18 08:00:06+00:00
7  -4.64269  53.32148  2013-09-19 08:00:06+00:00
10 -5.49176  52.48326  2013-09-20 08:00:25+00:00
13 -5.52468  49.98484  2013-09-21 08:00:37+00:00
16 -5.06282  50.15300  2013-09-22 08:00:06+00:00
19 -5.06294  50.15299  2013-09-23 08:00:37+00:00
22 -5.06286  50.15300  2013-09-24 08:00:06+00:00
25 -5.76222  49.25864  2013-09-25 08:00:06+00:00
28 -7.12106  47.42763  2013-09-26 08:00:06+00:00
31 -8.07914  44.77611  2013-09-27 08:00:06+00:00


In [26]:
october = daily[daily.time.dt.strftime('%Y%m%d').between('20131001', '20131031')]
print(october[:30])

          lon       lat                       time
43   -9.41521  38.69308  2013-10-01 08:00:06+00:00
45   -9.41776  38.69164  2013-10-03 08:00:06+00:00
48   -9.49440  37.66372  2013-10-04 08:00:30+00:00
51  -10.27359  35.99179  2013-10-05 08:00:06+00:00
54  -11.39966  34.05845  2013-10-06 08:00:06+00:00
57  -13.12509  31.81570  2013-10-07 08:00:06+00:00
60  -14.77898  29.34342  2013-10-08 08:00:06+00:00
63  -15.42484  28.12566  2013-10-12 08:48:55+00:00
66  -16.88826  26.02410  2013-10-13 08:01:07+00:00
69  -18.87698  23.39465  2013-10-14 08:00:30+00:00
72  -21.19816  21.15771  2013-10-15 08:01:01+00:00
74  -23.10809  18.79714  2013-10-16 10:51:11+00:00
77  -24.93056  16.98295  2013-10-17 08:00:37+00:00
78  -24.99189  16.88646  2013-10-17 10:04:06+00:00
84  -25.16579  13.38832  2013-10-20 08:00:06+00:00
87  -25.08661  10.97498  2013-10-21 08:00:06+00:00
92  -25.88019   3.63357  2013-10-24 08:00:06+00:00
97  -29.10563  -3.45941  2013-10-27 08:00:06+00:00
98  -30.51343  -6.78203  2013-1

In [53]:
era5_dir = '/home/jseddon/python/elinca/era5'
# chose every nth value
nth = 8
for i, dp in enumerate(october.iterrows()):
    de = dp[1]
    print(f'{de.time.year}{de.time.month:02}{de.time.day:02}')
    # Load data
    day_dir = os.path.join(era5_dir, f'{de.time.year}', f'{de.time.month:02}',
                           f'{de.time.day:02}')
    file_prefix = f'ecmwf-era5_oper_an_sfc_{de.time.year}{de.time.month:02}{de.time.day:02}1200'
    mslp = iris.load_cube(os.path.join(day_dir, file_prefix + '.msl.nc'))
    mslp_atl = mslp.intersection(longitude=(-90, 10))
    mslp_atl.convert_units('hPa')
    u = iris.load_cube(os.path.join(day_dir, file_prefix + '.10u.nc'))
    u_atl = u.intersection(longitude=(-90, 10))
    u_atl_2deg = u_atl[0, ::nth, ::nth]
    u_atl_2deg.convert_units('knot')
    v = iris.load_cube(os.path.join(day_dir, file_prefix + '.10v.nc'))
    v_atl = v.intersection(longitude=(-90, 10))
    v_atl_2deg = v_atl[0, ::nth, ::nth]
    v_atl_2deg.convert_units('knot')
    # Plot data
    plt.figure(figsize=(12, 25))
    iplt.quiver(u_atl_2deg, v_atl_2deg)
    iplt.contour(mslp_atl[0], 20,
                     linewidths=0.5, colors='black', linestyles='-')
    ax = plt.gca()
    # ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent((-70.0, 10.0, -90.0, 60.0), crs=crs_latlon)
    ax.stock_img()
    ax.coastlines(resolution='10m')
    ax.gridlines()
    plt.plot(de.lon, de.lat, marker='o', markersize=7.0, markeredgewidth=2.5,
                     markerfacecolor='black', markeredgecolor='white',
                     transform=crs_latlon)
    # Save plot
    output_dir = '/home/jseddon/python/elinca/plots'
    output_file = f'img{i:03}.png'
    plt.savefig(os.path.join(output_dir, output_file))
    plt.close()

20131001
20131003
20131004
20131005
20131006
20131007
20131008
20131012
20131013
20131014
20131015
20131016
20131017
20131017
20131020
20131021
20131024
20131027
20131028
20131029
20131031


In [51]:
de

lon                      -9.41521
lat                       38.6931
time    2013-10-01 08:00:06+00:00
Name: 43, dtype: object